In [21]:
import torch
import torch.nn as nn
from compressai.layers import conv1x1,conv3x3, AttentionBlock, CrossMaskedConv2d
class TestNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_enc = nn.Conv2d(3, 32, 3, 1, padding=1)
        self.conv_cross = CrossMaskedConv2d(32, 64, 5, 1, padding=2, bias=True, padding_mode="zeros")
        self.conv_ep = nn.Conv2d(64, 64, 3, 1, padding=1)
    
    def forward(self, x):
        x = self.conv_enc(x)
        
        y_anchor = x + 0.5
        y_anchor[..., 0::2, 1::2] = 0
        y_anchor[..., 1::2, 0::2] = 0
        
        y_ctx = self.conv_cross(y_anchor)
        y_ctx[..., 0::2, 0::2] = 0
        y_ctx[..., 1::2, 1::2] = 0
        
        out = self.conv_ep(y_ctx)
        return out


In [22]:
model = TestNet()
for m in model.parameters():
    m.data.fill_(0.1)
model.train()
images = torch.randn(2, 3, 16, 16)
out = model(images)
loss = (1000 * -torch.log2(out)).mean()
loss.backward()

In [26]:
model.conv_ep.weight.grad[16,16,...]

tensor([[-0.3417, -0.4334, -0.3383],
        [-0.4434, -0.3761, -0.4320],
        [-0.3583, -0.4518, -0.3462]])